# Лабораторная работа 6. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент Стеклов М. А. БФИ2001
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

In [246]:
from typing import Any
from pyvis.network import Network
import networkx as nx
import numpy as np

In [332]:
def file2graph(filename="lab6_io_files/lab6_input.txt") -> nx.DiGraph:
    !cat {filename}
    res = []
    with open(filename, 'r') as f:
        for line in f:
            res.append(list(map(float, line.strip().split())))
    return nx.from_numpy_matrix(np.array(res), create_using=nx.DiGraph)

In [419]:
import copy
def johnson(graph: nx.DiGraph, start_node, finish_node) -> list[Any]:
    graph = copy.deepcopy(graph)
    if all(edge_data[2]["weight"] >= 0 for edge_data in graph.edges(data=True)):
        return nx.dijkstra_path(graph, start_node, finish_node)
    graph.add_node('S')
    for node in graph.nodes:
        if node == 'S':
            continue
        graph.add_edge('S', node, weight=0)
    _, h = nx.bellman_ford_predecessor_and_distance(graph, 'S')
    graph.remove_node('S')
    for edge_data in graph.edges(data=True):
        edge_data[2]['weight'] += h[edge_data[0]] - h[edge_data[1]]
    return nx.dijkstra_path(graph, start_node, finish_node)



In [420]:
from random import randint, choice, shuffle

def gen_graph(n: int, a: int=10, b: int=50, neg_a: int=-9, neg_b: int=-1, neg_count: int=5):
    g = nx.dorogovtsev_goltsev_mendes_graph(n).to_directed()
    for edge_data in g.edges(data=True):
        edge_data[2]["weight"] = randint(a, b)
    edges_list = list(g.edges())
    for i in range(neg_count):
        u, v = choice(edges_list)
        g[u][v]["weight"] = randint(neg_a, neg_b)
    while True:
        tmp = list(g.edges())[::]
        shuffle(tmp)
        flag = True
        for u, v in tmp:
            if (v, u) in g.edges():
                g.remove_edge(v, u)
                flag = False
                break
        if flag:
            break 
    return g
        
        

In [421]:
#g = gen_graph(4)
g = file2graph()

0 1 0 0 0 1 6 0 
0 0 -3 0 0 0 0 0 
8 0 0 -1 0 0 0 9 
0 5 0 0 0 7 0 0 
5 0 1 0 0 0 0 0 
0 8 4 0 0 0 0 0 
0 0 4 2 -3 0 0 0 
0 5 0 6 0 0 0 0 


In [422]:
nt = Network(height='500px', width='100%', bgcolor='#222222', font_color='white', notebook=True, directed=True)
nt.from_nx(g)

for edge in nt.get_edges():
    edge["value"] = edge["weight"]
nt.show('nx.html')

In [423]:
start_node, finish_node = map(int,
                              input("Введите начальную и конечную точку пути: ").split())
path = johnson(g, start_node, finish_node)

Введите начальную и конечную точку пути: 0 3


In [424]:
path_edges = {(path[i], path[i+1]) for i in range(len(path) - 1)}

In [425]:
nt = Network(height='500px', width='100%', bgcolor='#222222', font_color='white', notebook=True, directed=True)
nt.from_nx(g)
for edge in nt.get_edges():
    edge["value"] = edge["weight"]
    if (edge["from"], edge["to"]) in path_edges:
        edge["color"] = "red"
nt.show('nx.html')

### Вывод